### original test data

In [1]:
import pandas as pd
language = 'es'
from datasets import load_dataset
dataset = load_dataset('xnli', split='test',language=language)
df_test = pd.DataFrame(dataset)
df_test

Found cached dataset xnli (/home/s6amalia/.cache/huggingface/datasets/xnli/default-language=es/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


,premise,hypothesis,label
0,"Bien, ni estaba pensando en eso, pero estaba t...",No he vuelto a hablar con él.,2
1,"Bien, ni estaba pensando en eso, pero estaba t...",Estaba tan molesto que empecé a hablar con él ...,0
2,"Bien, ni estaba pensando en eso, pero estaba t...",Tuvimos una gran charla.,1
3,"Y pensé que era un privilegio, y todavía es, t...",No sabía que no era la única persona en el cam...,1
4,"Y pensé que era un privilegio, y todavía es, t...",Tenía la impresión de que era el único con ese...,0
...,...,...,...
5005,Davidson no debería adoptar la pronunciación d...,Davidson no debería hablar de manera que bone ...,0
5006,Davidson no debería adoptar la pronunciación d...,Sería mejor si Davidson rimara las palabras bo...,2
5007,La novela media de 200 000 palabras por 25 $ f...,Una novela de 200.000 palabras a 25 dólares es...,1
5008,La novela media de 200 000 palabras por 25 $ f...,Una novela de 200 000 palabras de 25 $ son 400...,2


### Test data with typo

In [2]:
DATA_DIR = r"/home/s6amalia/thesis/XNLI_noisy_dataset/"
df_typo = pd.read_table(DATA_DIR+'test_'+language+'_typos_0.05.tsv')
df_typo

,premise,hypothesis,label
0,"'Y pensé que era un privilegio, y todavía es, ...","'A todos nos dieron el mismo número exacto, si...",2
1,'así que no tengo muy claro por qué.','Estoy seguro de la razon por la cuál.',2
2,'El chica que me puede ayudar está en la otra ...,'La chica de al que necesito ayuda vive muy le...,0
3,'pero ellos estaban divididos acerca de quiéne...,'No podían ponerse de acuerdo sobe quién era u...,0
4,'pero ellos estaban divididos acerca de quiene...,'No podían estar de acuerdo con quién deberia ...,1
...,...,...,...
682,'Uno debe ser muy cauto a la hora de proponer ...,'es delicado introducir las nuevas etimologías.',1
683,'Davidson no debería adoptar la pronunciación ...,'Davidson no debería hablar de manera que bone...,0
684,'Davidson no debería adoptar la pronunciación ...,'Sería Mejor si Davidson rimara las palabras b...,2
685,'La novela media de 200 000 palabras por 25 $f...,'Una novela de 200 000 palabras de 25 $son 400...,2


### Find original test data corresponding to the typo dataset

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf_matrix1 = vectorizer.fit_transform(df_typo['hypothesis'])
tfidf_matrix2 = vectorizer.transform(df_test['hypothesis'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix1, tfidf_matrix2)

# Threshold for considering texts as similar; change it to find the best number of data
threshold = 0.81

# Find similar rows
similar_rows = []
for i in range(len(df_typo)):
    for j in range(len(df_test)):
        if cosine_sim[i, j] > threshold:
            similar_rows.append((i, j))
columns = df_test.columns
similar_df = []

for pair in similar_rows:
    similar_df.append(df_test.iloc[pair[1]])
df_test_1 = pd.DataFrame(similar_df)   

print('Found',len(df_test_1),'of', len(df_typo))

Found 682 of 687


### Number of typos in each sentences

In [6]:
import difflib
def typo_count(s1,s2):
    
    # Create a Differ object
    differ = difflib.Differ()

    # Compare the two texts line by line
    diff = list(differ.compare(s1.splitlines(), s2.splitlines()))
    num_changes = sum(1 for line in diff if line.startswith('-') or line.startswith('+'))
    return num_changes

In [7]:
typos = []
for i in range(len(df_test)):
    s2 = df_typo['premise'].iloc[i][1:-1]
    s1 = df_test['premise'].iloc[i]
    typos.append(typo_count(s1,s2))

IndexError: single positional indexer is out-of-bounds

In [8]:
typo_count(df_test['premise'].iloc[100],df_typo['premise'].iloc[100][1:-1])

2

### Predict labels and Accuracy

In [9]:
from sklearn.metrics import accuracy_score
import torch

In [10]:
from transformers import AutoTokenizer,XLMRobertaForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("/home/s6amalia/xlmroberta-xnli-"+language+".tk")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
## loading the fine-tuned model
model = XLMRobertaForSequenceClassification.from_pretrained("/home/s6amalia/xlmroberta-xnli-"+language+".pt")

In [12]:
def predict_labels(df):
    all_pred = []
    for i in range(len(df)):
        premise = df['premise'].iloc[i]
        hypothesis = df['hypothesis'].iloc[i]
        input = tokenizer(premise, hypothesis, truncation=True,padding=True, return_tensors="pt")
        output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
#         print(output)
        prediction = torch.softmax(output["logits"][0], -1).tolist()
        label_names = ["entailment", "neutral", "contradiction"]
        # prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
        all_pred.append(prediction.index(max(prediction)))
        print(int(i*100/len(df)),'%', end='\r')
    print(round( accuracy_score(all_pred, df['label'])*100,2))   
    return all_pred

In [13]:
all_pred = predict_labels(df_typo)

78.02


In [116]:
all_pred = predict_labels(df_test)

82.65


In [14]:
all_pred = predict_labels(df_test_1)

79.47
